# Data Wrangling and Time-Series Creation

This notebook builds a daily time series of lettuce demand (in pounds) per store from menu and recipe transaction tables. Steps: (1) join raw tables, (2) convert units, (3) aggregate to daily demand, (4) fill missing days.

## 1. Open Datasets

Import fragmented datasets into working environment

In [30]:
# adjust if necessary
path_ingredients = './data/ingredients.csv'
path_menu_items = './data/menu_items.csv'
path_menuitem = './data/menuitem.csv'
path_portion_uom_types = './data/portion_uom_types.csv'
path_pos_ordersale = './data/pos_ordersale.csv'
path_recipe_ingredient_assignments = './data/recipe_ingredient_assignments.csv'
path_recipe_sub_recipe_assignments = './data/recipe_sub_recipe_assignments.csv'
path_recipes = './data/recipes.csv'
path_store_restaurant = './data/store_restaurant.xlsx'
path_sub_recipe_ingr_assignments = './data/sub_recipe_ingr_assignments.csv'
path_sub_recipes = './data/sub_recipes.csv'

In [31]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

ingredients = pd.read_csv(path_ingredients)
menu_items = pd.read_csv(path_menu_items)
menuitem = pd.read_csv(path_menuitem)
portion_uom_types = pd.read_csv(path_portion_uom_types)
pos_ordersale = pd.read_csv(path_pos_ordersale)
recipe_ingredient_assignments = pd.read_csv(path_recipe_ingredient_assignments)
recipe_sub_recipe_assignments = pd.read_csv(path_recipe_sub_recipe_assignments)
recipes = pd.read_csv(path_recipes)
store_restaurant = pd.read_excel(path_store_restaurant)
sub_recipe_ingr_assignments = pd.read_csv(path_sub_recipe_ingr_assignments)
sub_recipes = pd.read_csv(path_sub_recipes)

## 2. Data Cleansing

Inspect data for anomalies (missing values, wrong data types, etc.)

### 2.1 store_restaurant

In [35]:
store_restaurant

,STORE_ADDRESS1,STORE_ADDRESS2,DISTRIBUTION_REGION,STORE_STATE,STORE_CITY,STORE_ZIP,STORE_TYPE,STORE_LOYALTY_FLAG,STORE_NUMBER
0,Ninth Street,,Western,California,Berkeley,94710,Traditional,N,46673
1,Shattuck Sq,Shop #6,Western,California,Berkeley,94704,Traditional,Y,4904
2,Myrtle Avenue,,Northeast,New York,Ridgewood,11385,Traditional,Y,12631
3,Whitney Avenue,,Northeast,New York,Elmhurst,11373,Traditional,Y,20974


In [36]:
store_restaurant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   STORE_ADDRESS1       4 non-null      object
 1   STORE_ADDRESS2       4 non-null      object
 2   DISTRIBUTION_REGION  4 non-null      object
 3   STORE_STATE          4 non-null      object
 4   STORE_CITY           4 non-null      object
 5   STORE_ZIP            4 non-null      int64 
 6   STORE_TYPE           4 non-null      object
 7   STORE_LOYALTY_FLAG   4 non-null      object
 8   STORE_NUMBER         4 non-null      int64 
dtypes: int64(2), object(7)
memory usage: 420.0+ bytes


### 2.2 pos_ordersale

In [38]:
pos_ordersale.head()

,MD5KEY_ORDERSALE,ChangeReceived,OrderNumber,TaxInclusiveAmount,TaxAmount,MealLocation,TransactionId,StoreNumber,date
0,00BCB738A1F814DB960C25012E9913DF,0.00,67,0,0.56,0,67,46673,15-03-05
1,03438C257BC2DD0037F35EC8BFC0C8E2,3.46,39,0,0.54,0,39,12631,15-03-05
2,04AEF56CF3130AC1EFAD41C470F5245D,0.00,81,0,0.69,0,79,46673,15-03-05
3,04BBFAD4E73C30EE2EC3B4E52509637C,0.00,159,0,1.16,0,159,12631,15-03-05
4,061AE382DC127057E8ADBBF869CFB361,0.00,51,0,0.00,0,51,12631,15-03-05


### 2.3 menuitem

In [40]:
menuitem.head()

,MD5KEY_MENUITEM,MD5KEY_ORDERSALE,CategoryDescription,DepartmentDescription,Description,StoreNumber,TaxInclusiveAmount,TaxAmount,AdjustedPrice,DiscountAmount,Price,Quantity,PLU,Id,date
0,001E0224E4EDA63D6FED96D59E0C1E76,072D3B76BE58E4553A373AF3206D2CA5,Footlong,Footlong Sub,Chickn Strips FtLong,46673,0,0.6975,7.75,0.00,7.75,1,111000041,326,15-03-05
1,003E4E20196B2F5BDD33CE06D2943EF4,14811D4F3458058007B0B6908C2DA393,Fountain,Fountain Drink 21 oz.,21oz Fountain Drink,46673,0,0.1530,1.70,0.00,1.70,1,212000126,280,15-03-05
2,012A55E5551C95A559A7B074AE85117E,22FA6C39B7A15ABB6EB568E66A6B30E1,6 Inch,6 Inch Sub,B.M.T. 6 inch,12631,0,0.3230,3.64,0.61,4.25,1,121000008,8,15-03-05
3,01B07F1C6DC6945796571FC78735DA16,915B69612845BBD974C39AC105F5CA8C,6 Inch,6 Inch Sub,Veggie Delite 6 inch,12631,0,0.3328,3.75,0.00,3.75,1,121000001,91,15-03-05
4,02DE19974DE5304BF243A1C2EC639495,CD6386C802450EDCF13A3D112D1F86EC,Add On,Add On 6in Portion,Avocado Add6in,12631,0,0.0443,0.50,0.00,0.50,1,175000062,564,15-03-05


### 2.4 menu_items

In [42]:
menu_items.head()

,MenuItemName,MenuItemDescription,PLU,MenuItemId,RecipeId
0,FtL/Ham,Ham FtLong,111000004,1,6
1,Six/Ham,Ham 6 inch,121000004,2,7
2,FfB/Ham,Ham FtFbd,112000004,3,2
3,fBd/Ham,Ham FlatBd,122000004,4,8
4,Sld/Ham,Ham Salad,131000004,5,9


### 2.5 recipes

In [44]:
recipes.head()

,RecipeName,RecipeDescription,RecipeId
0,FtL/Ham,Ham FtLong,1
1,FfB/Ham,Ham FtFbd,2
2,Six/Ham,Ham 6 inch,3
3,fBd/Ham,Ham FlatBd,4
4,Sld/Ham,Ham Salad,5


### 2.6 recipe_ingredient_assignments

In [46]:
recipe_ingredient_assignments.head()

,RecipeId,IngredientId,Quantity
0,2,7,4
1,6,7,4
2,7,7,2
3,8,7,2
4,9,7,2


### 2.7 recipe_sub_recipe_assignments

In [48]:
recipe_sub_recipe_assignments.head()

,RecipeId,SubRecipeId,Factor
0,2,3,1
1,2,9,1
2,2,10,1
3,2,12,1
4,6,1,1


### 2.8 sub_recipes

In [50]:
sub_recipes.head()

,SubRecipeName,SubRecipeDescription,SubRecipeId
0,Std Footlong Sub bread,Std Footlong Sub Bread,1
1,Std 6 Inch Sub bread,Std 6 Inch Sub Bread,2
2,Std Footlong Flatbread Bread,Std Footlong Flatbread Bread,3
3,Std 6 Inch Flatbread bread,Std 6 Inch Flatbread bread,4
4,"Std Salad Lettuce, bowl, & dressing","Std Salad Lettuce, Bowl & Dressing",5


### 2.9 sub_recipe_ingr_assignments

In [52]:
sub_recipe_ingr_assignments.head()

,SubRecipeId,IngredientId,Quantity
0,1,1,1
1,1,135,2
2,1,138,1
3,1,146,1
4,1,157,1


### 2.10 ingredients

In [54]:
ingredients.head()

,IngredientName,IngredientShortDescription,IngredientId,PortionUOMTypeId
0,Bread ...,Bread Stick 12 inch/30cm,1,11.0
1,Bacon ...,Bacon,2,11.0
2,Chicken Strips ...,Chicken Strips,3,15.0
3,Chicken. oven roasted patty ...,"Chicken, single piece",4,15.0
4,Cold Cut Combo ...,Cold Cut Combo,5,15.0


### 2.11 portion_uom_types

In [56]:
portion_uom_types.head()

,PortionTypeDescription,PortionUOMTypeId
0,Each,11
1,Fluid Ounce,12
2,Gram,13
3,Milliliter,14
4,Ounce,15


## 3. Time-Series Extraction

### 3.1 Finding relevant IngredientIds

In [59]:
pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [60]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [61]:
lettuce_id_query = """SELECT 
    i.IngredientName,
    i.IngredientShortDescription,
    i.IngredientId,
    put.PortionTypeDescription
FROM 
    ingredients i
    INNER JOIN portion_uom_types put ON i.PortionUOMTypeId = put.PortionUOMTypeId
WHERE 
    (LOWER(i.IngredientName) LIKE '%lettuce%' OR LOWER(i.IngredientShortDescription) LIKE '%lettuce%');
"""

lettuce_ingredients = pysqldf(lettuce_id_query)
lettuce_ingredients

,IngredientName,IngredientShortDescription,IngredientId,PortionTypeDescription
0,Lettuce ...,Lettuce,27,Ounce
1,Lettuce - Metric ...,Lettuce - Metric,291,Gram


We seem to have two relevant entries, one in Ounce and one in Gram. In the following, we will use Gram as standard measure and convert any Ounce entries to grams. For that, we will use the conversion rate 1 Ounce = 28.3495 Grams.

### 3.2 Extraction of Store Ids

In [64]:
store_numbers_query = """
SELECT 
    STORE_CITY || ' ' || STORE_ADDRESS1 AS name,
    STORE_NUMBER AS store_id
FROM store_restaurant
"""
stores = pysqldf(store_numbers_query)
stores

,name,store_id
0,Berkeley Ninth Street ...,46673
1,Berkeley Shattuck Sq ...,4904
2,Ridgewood Myrtle Avenue ...,12631
3,Elmhurst Whitney Avenue,20974


In [65]:
# Store ID to Name mapping
store_id_to_name = {
    46673: 'California 1 (ID:46673)',
    4904 : 'California 2 (ID:4904)',
    12631: 'New York 1 (ID:12631)',
    20974: 'New York 2 (ID:20974)'
}

In [66]:
# Renaming the 'name' column using the store_id_to_name dictionary
stores['name'] = stores['store_id'].map(store_id_to_name)

# Show the updated DataFrame
print(stores)

                      name  store_id
0  California 1 (ID:46673)     46673
1   California 2 (ID:4904)      4904
2    New York 1 (ID:12631)     12631
3    New York 2 (ID:20974)     20974


### 3.3 SQL Wrapper

Loop through the data by IngredientId and StoreNumber.

Potential values in metric system are converted into ounces by dividing them by 28.34953.

In [69]:
from functools import reduce
from pandasql import sqldf  # Assuming you've set up pandasql to run queries on DataFrames

# SQL template with a placeholder for store_id
ts_extraction_query_template = """
SELECT  
    ts_date, 
    SUM(ts_quantity) AS total_quantity
FROM (
    SELECT 
        m.date AS ts_date, 
        SUM(m.Quantity * ria.Quantity) AS ts_quantity 
    FROM 
        menuitem m
        INNER JOIN menu_items mi ON m.Id = mi.MenuItemId AND m.PLU = mi.PLU
        INNER JOIN recipe_ingredient_assignments ria ON mi.RecipeId = ria.RecipeId
    WHERE 
        ria.IngredientId = '27' AND m.StoreNumber = {store_id}
    GROUP BY 
        m.date

    UNION ALL

    SELECT 
        m.date AS ts_date, 
        SUM(m.Quantity * ria.Quantity) / 28.34953 AS ts_quantity 
    FROM 
        menuitem m
        INNER JOIN menu_items mi ON m.Id = mi.MenuItemId AND m.PLU = mi.PLU
        INNER JOIN recipe_ingredient_assignments ria ON mi.RecipeId = ria.RecipeId
    WHERE 
        ria.IngredientId = '291' AND m.StoreNumber = {store_id}
    GROUP BY 
        m.date

    UNION ALL

    SELECT 
        m.date AS ts_date, 
        SUM(m.Quantity * sri.Quantity * rsa.Factor) AS ts_quantity 
    FROM 
        menuitem m
        INNER JOIN menu_items mi ON m.Id = mi.MenuItemId AND m.PLU = mi.PLU
        INNER JOIN recipe_sub_recipe_assignments rsa ON mi.RecipeId = rsa.RecipeId
        INNER JOIN sub_recipe_ingr_assignments sri ON rsa.SubRecipeId = sri.SubRecipeId
    WHERE 
        sri.IngredientId = '27' AND m.StoreNumber = {store_id}
    GROUP BY 
        m.date

    UNION ALL

    SELECT 
        m.date AS ts_date, 
        SUM(m.Quantity * sri.Quantity * rsa.Factor) / 28.34953 AS ts_quantity 
    FROM 
        menuitem m
        INNER JOIN menu_items mi ON m.Id = mi.MenuItemId AND m.PLU = mi.PLU
        INNER JOIN recipe_sub_recipe_assignments rsa ON mi.RecipeId = rsa.RecipeId
        INNER JOIN sub_recipe_ingr_assignments sri ON rsa.SubRecipeId = sri.SubRecipeId
    WHERE 
        sri.IngredientId = '291' AND m.StoreNumber = {store_id}
    GROUP BY 
        m.date
) AS combined
GROUP BY 
    ts_date
ORDER BY 
    ts_date ASC;
"""

# List to collect each store's time series DataFrame
time_series_data = []

# Loop directly through the stores DataFrame
for index, row in stores.iterrows():
    ts_extraction_query = ts_extraction_query_template.format(store_id=row['store_id'])
    result = sqldf(ts_extraction_query)
    result.rename(columns={'total_quantity': row['name']}, inplace=True)
    time_series_data.append(result)

# Merge all DataFrames on 'ts_date'
final_df = reduce(lambda left, right: pd.merge(left, right, on='ts_date', how='outer'), time_series_data)

# Display the final DataFrame
print(final_df)

      ts_date  California 1 (ID:46673)  California 2 (ID:4904)  \
0    15-03-05                      152                     NaN   
1    15-03-06                      100                     NaN   
2    15-03-07                       54                     NaN   
3    15-03-08                      199                     NaN   
4    15-03-09                      166                     NaN   
..        ...                      ...                     ...   
98   15-06-11                      151                   256.0   
99   15-06-12                      134                   223.0   
100  15-06-13                       96                   216.0   
101  15-06-14                      138                   350.0   
102  15-06-15                      244                   338.0   

     New York 1 (ID:12631)  New York 2 (ID:20974)  
0                      232                    NaN  
1                      227                    4.0  
2                      181                    NaN  

### 3.4 Extracting store-wise Series

In the following, the different time series will be abbreviated as:
(1) y for Aggregate Demand of all four stores
(2) y_44673 for 'California 1 (ID:46673)'
(3) y_4904 for 'California 2 (ID:4904)'
(4) y_12631 for 'New York 1 (ID:12631)'
(5) y_20974 for 'New York 2 (ID:20974)'

In [72]:
# Convert ts_date column to datetime if not already done
final_df['ts_date'] = pd.to_datetime(final_df['ts_date'], format='%y-%m-%d')  # Adjust format if needed

# Set ts_date as index
final_df.set_index('ts_date', inplace=True)

In [73]:
# Replace NaN values with 0 in the entire DataFrame
final_df.fillna(0, inplace=True)

In [74]:
final_df['total'] = final_df['California 1 (ID:46673)'] + final_df['California 2 (ID:4904)'] + final_df['New York 1 (ID:12631)'] + final_df['New York 2 (ID:20974)']

In [75]:
# Replace NaN values with 0 in the entire DataFrame
final_df.fillna(0, inplace=True)

In [76]:
final_df['total'] = final_df['California 1 (ID:46673)'] + final_df['California 2 (ID:4904)'] + final_df['New York 1 (ID:12631)'] + final_df['New York 2 (ID:20974)']

In [77]:
final_df.head()

,California 1 (ID:46673),California 2 (ID:4904),New York 1 (ID:12631),New York 2 (ID:20974),total
ts_date,,,,,
2015-03-05,152,0.0,232,0.0,384.0
2015-03-06,100,0.0,227,4.0,331.0
2015-03-07,54,0.0,181,0.0,235.0
2015-03-08,199,0.0,221,0.0,420.0
2015-03-09,166,0.0,229,0.0,395.0


### 3.5 Export Data for Forecasting

In [112]:
# --- Export wrangled data for forecasting ---
import os

def save_output(out_path="data/processed/final_demand.parquet"):
    os.makedirs("data/processed", exist_ok=True)
    final_df.to_parquet(out_path, index=False)
    print(f" Exported {len(final_df):,} rows to data/processed/")

# Notebook call (no argparse; safe in Jupyter)
save_output()

 Exported 103 rows to data/processed/


In [110]:
# Script entry-point (used only when this file is executed as a .py script)
if __name__ == "__main__":
    import sys
    # If running inside Jupyter, skip argparse to avoid '-f kernel.json' error
    if "ipykernel" not in sys.modules:
        import argparse
        p = argparse.ArgumentParser()
        p.add_argument("--out", default="data/processed/final_demand.parquet")
        args = p.parse_args()
        save_output(args.out)

usage: ipykernel_launcher.py [-h] [--out OUT]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\max-u.MTU-SFC\AppData\Roaming\jupyter\runtime\kernel-271e9ccb-fcd2-49a4-964d-2a888408f724.json


SystemExit: 2